# Exploring the use of spatial estimates of GHG emissions, nutrient application and water use

Converting enviormental impact estimates associated with crop and livestock production from Halpern et al., 2022, "The environmental footprint of global food production" https://doi.org/10.1038/s41893-022-00965-x

## Purpose

Takes the impact estimates from Halpern et al., by crop group and livestock category for the year 2017 and converts these to impact intensities, which can be applied to estimate impact intensities of consumption

## Methodology

Take GHG emissions, excess nutrient production and water use associated with crop groups and livestock, and divide by the production to convert into an emissions intensity

GHG emissions exclude LUC

June 2023: Global food system pressure data was downloaded from: https://knb.ecoinformatics.org/view/doi:10.5063/F1V69H1B

Excess N & P: "Excess N and P nutrients are combined for the assessments in the paper. These mapped data provide excess N and P data separately for all crops (synthetic fertilizers) and reared animals (manure/excretion). Units are tonnes excess N or P (includes, leaching, runoff, volatilization), with coordinate reference system as Gall-Peters equal area with a resolution of 36km2.". Also "tonnes of leached/runoff/volatilized N or P from crops (synthetic fertilizers) and farmed animals (manure/excretion)."

Crop impacts: "Pressure data for 26 crop categories, excluding the portion estimated to be used as animal feed. Pressures are provided per cell and include disturbance (km2eq); blue freshwater consumption (m3 water); excess nutrients (tonnes NP); and greenhouse gas emissions (tonnes CO2eq)."

Livestock impacts: "Pressure data for several categories of livestock incurred at the farm site and for crop, fodder, and fish oil/meal feed. Pressures are provided per cell and include disturbance (km2eq); blue freshwater consumption (m3 water); excess nutrients (tonnes NP); and greenhouse gas emissions (tonnes CO2eq)."

Production data is: "Mapped production data for crops, fisheries, livestock, mariculture. Data are in tonnes with units of production from FAO data. Coordinate reference system is Gall-Peters equal area with a resolution of 36km2."


Usage rights: Halpern et al. 2022 data is provided with the following usage rights: "This work is dedicated to the public domain under the Creative Commons Universal 1.0 Public Domain Dedication. To view a copy of this dedication, visit https://creativecommons.org/publicdomain/zero/1.0/"

## WIP - improvements

Use this section only if the notebook is not final.

Notable TODOs:

- Todo 1;
- Todo 2;


## Results

Describe and comment the most important results.

## Suggested next steps

State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import

We import all the required PYthon libraries

In [129]:
!pip install rasterio
!pip install rioxarray

     |████████████████████████████████| 53 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 973 kB 3.6 MB/s eta 0:00:01


In [131]:
# Data manipulation
import os
import pandas as pd
import geopandas as gpd
import numpy as np

# Visualization
import plotly
import matplotlib
import matplotlib.pyplot as plt
import rasterio as rio
import rioxarray as riox
import xarray

from rasterio.warp import calculate_default_transform, reproject, Resampling


## Data import

In [29]:
# Read in a list of crop super names

root_data_path = '/mnt/c/Users/mikeha/Work/Spatial data/Environmental footprint of global food production/'
crop_table_fname = root_data_path + 'SI_SPAM_crops_tbl.csv'
crop_table = pd.read_csv(crop_table_fname,quotechar='"')

spam_supers = np.unique(crop_table['SPAM_super'])


In [80]:
#Read grid cell areas for wgs84 grids (production)
dst_wgs84_grid_areas_km2 = rio.open(root_data_path + 'wgs84_grid_areas_km2.tif')

# Read the raster data
wgs84_grid_areas_km2 = dst_wgs84_grid_areas_km2.read(1)  # Assuming a single-band raster, change the index if needed

np.nanmax(wgs84_grid_areas_km2)

85.47965

## Parameter definition

In [89]:
# We set all relevant parameters for our notebook. (agrrements in naming convention).

#Data paths

prod_data_path = root_data_path + 'extra_production/'
impact_data_path = root_data_path + 'crops_food_raw/'

impact_km2_path = root_data_path + "crops_foods_raw_km2/"
impact_reprojected_km2_path = root_data_path + 'crops_food_raw_km2_wgs84/'
ghg_rate_path = root_data_path + 'ghg_per_t_product_food_wgs84/'
nutrient_rate_path = root_data_path + 'nutrient_per_t_product_food_wgs84/'

# Define the target CRS (WGS84)
target_crs = 'EPSG:4326'


## Data processing

In [52]:
c = spam_supers[0]
p_src = rio.open(prod_data_path + 'crop_'+c+'_production.tif')
print('production no data = ' + str(p_src.nodata))

ghg_src = rio.open(impact_data_path + 'gall_peter_farm_land_' + c + '_crop_produce_ghg_per_cell.tif')
print('ghg no data = ' + str(ghg_src.nodata))

nut_src = rio.open(impact_data_path + 'gall_peter_farm_land_' + c + '_crop_produce_nutrient_per_cell.tif')
print('nutrient no data = ' + str(nut_src.nodata))

production no data = -3.3999999521443642e+38
ghg no data = -3.3999999521443642e+38
Nutrient no data = -3.3999999521443642e+38


In [86]:
p_src.res
ghg_src.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -3.3999999521443642e+38,
 'width': 4736,
 'height': 3000,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Cylindrical_Equal_Area"],PARAMETER["standard_parallel_1",45],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(6000.0, 0.0, -14207430.316916062,
        0.0, -6000.0, 8999818.14485532)}

In [127]:
def reproject_impacts(source_raster_path,target_raster_path,impact_reprojected_km2_path,p_src,wgs84_grid_areas_km2):

    # Open the source raster
    with rio.open(source_raster_path) as src:
        
        # convert the impact values in src to an impact per unit area (km2)
        # impact pixel resolution is 6 x 6 and equal area- so divide by 36  
        src_data = src.read(1, masked = True)
        #src_data[src_data == src.nodata] = np.NaN
        src_km2 = src_data/36.0

        #Interim output of the area rate of impact in its original projection
        #with rio.open(impact_km2_path + 'test_bana.tif','w', **src.meta.copy()) as dst:
        #    dst.write(src_km2,indexes = 1)

        # Calculate the transformation parameters
        transform, width, height = calculate_default_transform(
            src_crs = src.crs,
            dst_crs = target_crs,
            width= src.width,height= src.height,
            left=src.bounds[0], bottom=src.bounds[1], right=src.bounds[2], top=src.bounds[3],
            resolution=p_src.res
        )
        
        # Create the target raster dataset
        # use p_src.width and height in order to make sure get the correct 
        kwargs = p_src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        print(kwargs)
        
        #create an empty array for the destination data
        #dst_km2 = None # np.array(p_src.shape,np.float32)

        with rio.open(impact_reprojected_km2_path, 'w', **kwargs) as dst:

            # Reproject the source raster to the target CRS
            reproject(
                source=src_km2,
                destination=rio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.bilinear
            )

            #convert to gridcell impact
            #dst_cell = dst_km2 * wgs84_grid_areas_km2

            #dst.write(dst_cell,1)
        
        with rio.open(impact_reprojected_km2_path) as dst_prj_km2:
            prj_data = dst_prj_km2.read(1, masked = True)
            prj_data = prj_data * np.ma.masked_where(np.ma.getmask(prj_data), wgs84_grid_areas_km2[0:prj_data.shape[0],:])
            with rio.open(target_raster_path, 'w', **kwargs) as dst_prj:
                dst_prj.write(prj_data, 1)

        #with rio.open(impact_reprojected_km2_path + 'test_bana.tif', 'w', **kwargs) as dst:
        #    dst.write(dst_cell/wgs84_grid_areas_km2,indexes = 1)

In [157]:
# source_raster_path = the equal area 36km2 impact data
# target_raster_path = the raster to be written out in wgs84
# match_raster_path = a raster to match the reprojection to
# impact_reprojected_km2_path = the reprojected raster in impact per km2
# wgs84_grid_areas = pixel areas (calculated in R and exported as a raster) 
def reproject_impacts_riox(source_raster_path,target_raster_path,match_raster_path,impact_reprojected_km2_path,wgs84_grid_areas_km2_path):
    #Open the source impact data
    src_datasetreader = rio.open(source_raster_path)
    src_xds = riox.open_rasterio(src_datasetreader, masked=True)
    #set the crs of src file
    src_xds.rio.write_crs(src_datareader.crs, inplace=True)

    #open and read the match raster file (wgs84)
    match_datasetreader = rio.open(match_raster_path)
    match_xds = riox.open_rasterio(match_datasetreader, masked=True)
    match_xds.rio.write_crs("EPSG:4326",inplace=True)

    wgs84_pixel_areas_xds = riox.open_rasterio(wgs84_grid_areas_km2_path)#,engine="rasterio")
    
    #Convert src impact to a per km2 rate by dividing by the cell area
    src_xds_km2 = src_xds/36.0

    #reproject
    src_match_xds_km2 = src_xds_km2.rio.reproject_match(match_xds,resampling=Resampling.bilinear)
    
    #export the reprojected per km2 raster
    src_match_xds_km2.rio.to_raster(impact_reprojected_km2_path)

    #convert to per cell impacts and export
    src_match_xds = src_match_xds_km2 * wgs84_pixel_areas_xds
    src_match_xds.rio.to_raster(target_raster_path)


In [94]:
if os.path.exists(impact_km2_path) == False: os.mkdir(impact_km2_path)
if os.path.exists(impact_reprojected_km2_path) == False: os.mkdir(impact_reprojected_km2_path)
if os.path.exists(ghg_rate_path) == False: os.mkdir(ghg_rate_path)
if os.path.exists(nutrient_rate_path) == False: os.mkdir(nutrient_rate_path)


In [159]:

#loop over crop types
#for c in spam_supers:


#read in the impact data
# Source raster path (Gall-Peters CRS)
ghg_source_raster_path = impact_data_path + 'gall_peter_farm_land_' + c + '_crop_produce_ghg_per_cell.tif'


# Target raster path (WGS84 CRS)
ghg_target_raster_path = ghg_rate_path + 'ghg_per_t_product_' + c + '.tif'


ghg_reprojected_km2_raster_path = impact_reprojected_km2_path + 'rio_ghg_per_km2_' + c + '.tif'

# reproject_impacts(ghg_source_raster_path,
#                   ghg_target_raster_path,
#                   ghg_reprojected_km2_raster_path,
#                   p_src,
#                   wgs84_grid_areas_km2)



# Target raster path (WGS84 CRS) rioxarray
rxa_ghg_target_raster_path = ghg_rate_path + 'rxa_ghg_per_t_product_' + c + '.tif'
rxa_ghg_target_km2_raster_path = impact_reprojected_km2_path + 'rxa_ghg_per_km2_' + c + '.tif'
match_raster_path = prod_data_path + 'crop_'+c+'_production.tif'


reproject_impacts_riox(ghg_source_raster_path,
                        rxa_ghg_target_raster_path,
                        prod_data_path + 'crop_'+c+'_production.tif',
                        rxa_ghg_target_km2_raster_path,
                        '/mnt/c/Users/mikeha/Work/Spatial data/Environmental footprint of global food production/wgs84_grid_areas_km2.tif')




/home/mikeha/.local/lib/python3.8/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


## References

Report here relevant references:

1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2